In [1]:

import igl
import meshio as mio
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import subprocess
import scipy
import h5py


In [64]:
workspace_path = "/Users/jiachengdai/Desktop/algebraic-contours/build/test_pipeline_icosphere/"

interior_matix = scipy.io.mmread(workspace_path+'CT_interior_constraint_matrix.txt') 
edge_end_point_matrix = scipy.io.mmread(workspace_path+'CT_edge_endpoint_constraint_matrix_eliminated.txt') 
edge_mid_point_matrix = scipy.io.mmread(workspace_path+'CT_edge_midpoint_constraint_matrix.txt')

full_matrix = scipy.sparse.vstack((interior_matix, edge_end_point_matrix, edge_mid_point_matrix))

local2global = np.loadtxt(workspace_path + "icosphere_tri_to_tet_v_map.txt").astype(np.int32)

m = mio.read(workspace_path + "icosphere_initial_tetmesh.msh")


In [65]:
v = m.points
cells_ho = m.cells_dict['tetra20']
cells = cells_ho[:,:4]
bd_f = igl.boundary_facets(cells)
bd_v = np.unique(bd_f)
l2g_r = np.flip(local2global)

In [66]:
# build P_T, left mul x  i.e. P_T x
# P right mul C i.e. C P
constrained = [False for i in range(v.shape[0])]
for i in range(l2g_r.shape[0]):
    constrained[l2g_r[i]] = True
for i in range(bd_v.shape[0]):
    constrained[bd_v[i]] = True
unconstrained_v = []
for i in range(len(constrained)):
    if not constrained[i]:
        unconstrained_v.append(i)

P_T = [-1 for i in range(v.shape[0])]
for i in range(l2g_r.shape[0]):
    P_T[i] = l2g_r[i]
for i in range(bd_v.shape[0]):
    P_T[l2g_r.shape[0] + i] = bd_v[i]
for i in range(len(unconstrained_v)):
    P_T[l2g_r.shape[0] + bd_v.shape[0] + i] = unconstrained_v[i]

assert all(P_T[i]>-1 for i in range(len(P_T)))

P_T_trip_row = []
P_T_trip_col = []
P_T_trip_value = [1 for i in range(len(P_T) * 3)]

for i in range(len(P_T)):
    P_T_trip_row.append(i*3+0)
    P_T_trip_row.append(i*3+1)
    P_T_trip_row.append(i*3+2)
for i in range(len(P_T)):
    P_T_trip_col.append(P_T[i]*3+0)
    P_T_trip_col.append(P_T[i]*3+1)
    P_T_trip_col.append(P_T[i]*3+2)



In [67]:
# add direchlet constraints
di_row = [i for i in range(bd_v.shape[0])]
di_col = [bd_v[i] for i in range(bd_v.shape[0])]
di_value = [1.0 for i in range(bd_v.shape[0])]

di_matrix = scipy.sparse.coo_array((di_value, (di_row,di_col)), shape=(bd_v.shape[0], v.shape[0]))

# expand constraint on tetmesh
full_matrix_coo = full_matrix.tocoo(True)
full_matrix_row = full_matrix_coo.row
full_matrix_col = full_matrix_coo.col
full_matrix_data = full_matrix_coo.data

full_matrix_tet_row = full_matrix_row
full_matrix_tet_col = np.array([local2global[full_matrix_col[i]] for i in range(full_matrix_col.shape[0])])
full_matrix_tet_data = full_matrix_data

full_matrix_tet = scipy.sparse.coo_array((full_matrix_tet_data, (full_matrix_tet_row, full_matrix_tet_col)), shape=(full_matrix.shape[0], v.shape[0]))

C_matrix = scipy.sparse.vstack((full_matrix_tet, di_matrix))

In [80]:
len(P_T_trip_row)

24855

In [83]:
C_matrix.shape

(1599, 8285)

In [84]:
v.shape

(8285, 3)

In [89]:
xxx = np.unique(np.array(P_T_trip_col))
xxxx = np.unique(np.array(P_T_trip_row))

In [90]:
xxxx.shape

(24855,)

In [ ]:
M = scipy.io.mmread("test_M.")

In [3]:
a = np.array([[1, 2], [1, 2], [3, 2]])
np.unique(a, axis=0)

array([[1, 2],
       [3, 2]])